In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
cd '/content/drive/MyDrive/딥러닝 과제/Pokemon'

/content/drive/MyDrive/딥러닝 과제/Pokemon


In [30]:
!ls

checkpoint.pt	     Fake	    Model
checkpoint_VGG16.pt  image	    requirements.txt
checkpoint_VGG19.pt  loss_plot.png  splitted


In [31]:
import os
import shutil
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import random
import cv2
import glob
from PIL import Image
import PIL.ImageOps
from os import remove

## 파일 확장자 확인

In [ ]:
## 파일 확장자 중에 png가 아닌 파일이 있음!
# 참고 : https://appia.tistory.com/536
# 참고 : https://codeomni.tistory.com/435
# folder_path는 image 입니다.
folder_path = './image'
# file_names에는 위의 폴더 내부에 있는 이미지 들의  배열이 저장됩니다.
folder_names = os.listdir(folder_path)
# file_path에 원본 사진이 있는 폴더를 정해줍니다.

for _, name in enumerate(folder_names):

  file_path = folder_path + '/' + name
  # file_names에는 위의 폴더 내부에 있는 이미지 배열이 저장됩니다.
  file_names = os.listdir(file_path)
  # file_names의 길이를 저장합니다. 만약 50장의 원본 이미지가 있다면, 이 변수는 50이 될 것입니다.
  total_origin_image_num = len(file_names)
  path = file_path

  format = [".jpg",".png",".jpeg","bmp",".JPG",".PNG","JPEG","BMP"] # 지원하는 포맷확장자 나열
  for (path,dirs,files) in os.walk(path):
      for file in files:
          if file.endswith(tuple(format)):
              pass
          else:
              print(path)
              print("InValid 파일을 제거합니다.",file)
              os.remove(path + '/' + file)

## 데이터 분할

In [ ]:
import os
import shutil
 
original_dataset_dir = './image'
classes_list = os.listdir(original_dataset_dir) 

# Fake로 경로를 바꾸었음
base_dir = './Fake/split' #이미 splitted 폴더가 존재하면 돌아가지 않아요
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:     
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

## 데이터 분할과 클래스별 데이터의 수

In [ ]:
import math
 
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Abra ):  21
Validation size( Abra ):  7
Test size( Abra ):  7
Train size( Aerodactyl ):  21
Validation size( Aerodactyl ):  7
Test size( Aerodactyl ):  7
Train size( Arcanine ):  24
Validation size( Arcanine ):  8
Test size( Arcanine ):  8
Train size( Alakazam ):  15
Validation size( Alakazam ):  5
Test size( Alakazam ):  5
Train size( Arbok ):  24
Validation size( Arbok ):  8
Test size( Arbok ):  8
Train size( Articuno ):  27
Validation size( Articuno ):  9
Test size( Articuno ):  9
Train size( Alolan Sandslash ):  24
Validation size( Alolan Sandslash ):  8
Test size( Alolan Sandslash ):  8
Train size( Golduck ):  27
Validation size( Golduck ):  9
Test size( Golduck ):  9
Train size( Gastly ):  26
Validation size( Gastly ):  8
Test size( Gastly ):  8
Train size( Geodude ):  21
Validation size( Geodude ):  7
Test size( Geodude ):  7
Train size( Grimer ):  26
Validation size( Grimer ):  8
Test size( Grimer ):  8
Train size( Goldeen ):  28
Validation size( Goldeen ):  9
Test 

## Data Augmentation

### Data Augmentation Val에서는 사용하지 않음
- https://datascience.stackexchange.com/questions/41422/when-using-data-augmentation-is-it-ok-to-validate-only-with-the-original-images

In [ ]:
# 참고 사이트 : https://hipolarbear.tistory.com/19
# # ### Train 폴더의 이름 찾기
# # # folder_path는 train folder 입니다.
# folder_path = './splitted/train'
# # file_names에는 위의 폴더 내부에 있는 이미지 배열이 저장됩니다.
# folder_names = os.listdir(folder_path)
# # file_names의 길이를 저장합니다. 만약 50장의 원본 이미지가 있다면, 이 변수는 50이 될 것입니다.
# total_origin_image_num = len(folder_names)
# # file_path에 원본 사진이 있는 폴더를 정해줍니다.
# file_path = './splitted/train' + '/' + folder_names[0]
# # file_names에는 위의 폴더 내부에 있는 이미지 배열이 저장됩니다.
# file_names = os.listdir(file_path)
# # file_names의 길이를 저장합니다. 만약 50장의 원본 이미지가 있다면, 이 변수는 50이 될 것입니다.
# total_origin_image_num = len(file_names)

### Augmentation 사진 추가하기

In [32]:
def Add_Picture(mode, num):

  # Augmentation 모드를 지정합니다.
  # test는 아직 알지 못하는 데이터
  ls = ['train']

  ratio = [1]

  # folder_path는 train folder 입니다.
  folder_path = './splitted/' + ls[mode]
  # file_names에는 위의 폴더 내부에 있는 이미지 배열이 저장됩니다.
  folder_names = os.listdir(folder_path)
  # file_names의 길이를 저장합니다. 만약 50장의 원본 이미지가 있다면, 이 변수는 50이 될 것입니다.
  total_origin_image_num = len(folder_names)

  for _, name in enumerate(folder_names):
    
    # file_path에 원본 사진이 있는 폴더를 정해줍니다.
    file_path = './splitted/' + ls[mode]  + '/' + name
    # file_names에는 위의 폴더 내부에 있는 이미지 배열이 저장됩니다.
    file_names = os.listdir(file_path)
    # file_names의 길이를 저장합니다. 만약 50장의 원본 이미지가 있다면, 이 변수는 50이 될 것입니다.
    total_origin_image_num = len(file_names)


    # 사진 묶음 씩 Augemntation 하기
    num_augmented_images = num * ratio[mode]
    # 몇 번의 augmentation이 일어났는지를 확인해주기 위해 선언되었습니다.
    augment_cnt = 1

    for i in range(1, int(num_augmented_images) + 1):

        change_picture_index = random.randrange(1, total_origin_image_num-1)
        # print(change_picture_index)
        # print(file_names[change_picture_index])
        file_name = file_names[change_picture_index]
        
        origin_image_path = file_path + '/' + file_name
        # print(origin_image_path)
        image = Image.open(origin_image_path)
        random_augment = random.randrange(1,4)

        if(random_augment == 1):
            #이미지 좌우 반전
            # print("invert")
            inverted_image = image.transpose(Image.FLIP_LEFT_RIGHT)
            inverted_image.save(f'{file_path}' + "/" + 'inverted_' + str(augment_cnt) + '.png')
            
        elif(random_augment == 2):
            #이미지 기울이기
            # print("rotate")
            rotated_image = image.rotate(random.randrange(-20, 20))
            rotated_image.save(f'{file_path}' + "/" + 'rotated_' + str(augment_cnt) + '.png')
            
        elif(random_augment == 3):
            #노이즈 추가하기
            img = cv2.imread(origin_image_path)
            # print("noise")
            row,col,ch= img.shape
            mean = 0
            var = 0.1
            sigma = var**0.5
            gauss = np.random.normal(mean,sigma,(row,col,ch))
            gauss = gauss.reshape(row,col,ch)
            noisy_array = img + gauss
            noisy_image = Image.fromarray(np.uint8(noisy_array)).convert('RGB')
            noisy_image.save(f'{file_path}' + "/" + 'noiseAdded_' + str(augment_cnt) + '.png')
            
        augment_cnt += 1

### Augmentation 실행

In [33]:
# 추가할 전체 사진의 수
pic_num = 110
# Train
Add_Picture(0, pic_num)
# Val, Test 둘 다 안함

## 파일을 다시 삭제하는 방법 : Val에는 일반적으로 augmentaion이 쓰이지 않음

In [ ]:
!ls

checkpoint.pt	     checkpoint_VGG19.pt  loss_plot.png  requirements.txt
checkpoint_VGG16.pt  image		  Model		 splitted


In [ ]:
## 파일 확장자 중에 inverted_, rotated_, noiseAdded_가 들어있는 파일 삭제
# 참고 : https://blog.naver.com/PostView.nhn?blogId=hankrah&logNo=221829206710
# 참고 : https://security-nanglam.tistory.com/429
# folder_path는 image 입니다.
folder_path = './splitted/train'
# file_names에는 위의 폴더 내부에 있는 이미지 들의  배열이 저장됩니다.
folder_names = os.listdir(folder_path)
# file_path에 원본 사진이 있는 폴더를 정해줍니다.

for _, name in enumerate(folder_names):

  file_path = folder_path + '/' + name
  # file_names에는 위의 폴더 내부에 있는 이미지 배열이 저장됩니다.
  file_names = os.listdir(file_path)
  # file_names의 길이를 저장합니다. 만약 50장의 원본 이미지가 있다면, 이 변수는 50이 될 것입니다.
  total_origin_image_num = len(file_names)
  path = file_path

  format = ["inverted_","rotated_","noiseAdded_"] # 지원하는 포맷확장자 나열
  for (path,dirs,files) in os.walk(path):
      for file in files:
          if file.startswith(tuple(format)):
              print(path)
              print("InValid 파일을 제거합니다.",file)
              os.remove(path + '/' + file)
          else:
              pass